In [7]:
# save as prep.py and run:  python prep.py
from pathlib import Path
import pandas as pd, numpy as np, tqdm

SRC = Path("../../data/raw/s&p500/1D")          # adjust if the folder moved / renamed
DST = Path("../../data/data_prepared/train")
DST.mkdir(parents=True, exist_ok=True)

for csv in tqdm.tqdm(list(SRC.glob("*.csv"))):
    df = pd.read_csv(csv, parse_dates=["Date"])
    series = (df["Close"]
              .ffill()              # forward-fill any isolated NaNs
              .clip(lower=1e-6))    # prevent log(0)
    out = pd.DataFrame({
        "Date": df["Date"],
        "value": np.log(series.values, dtype=np.float32)  # <-- log-price
    })
    out.to_csv(DST / csv.name, index=False)


100%|██████████| 497/497 [01:47<00:00,  4.61it/s]


In [ ]:
# my_config.py  (copy from configs/fine_tuning.py and edit)
batch_size = 128          # fits on ~12 GB
total_epochs = 40         # enough for ~80 k updates on S&P 500 daily
learning_rate = 5e-5      # a bit gentler on a smaller dataset
